<a href="https://colab.research.google.com/github/Ul-tima/ChatGPTCoverLetter/blob/main/ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  --quiet openai python-dotenv

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd /content/gdrive/MyDrive/ChatGPT/

Mounted at /content/gdrive
/content/gdrive/MyDrive/ChatGPT


In [8]:
import openai
import os
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [47]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.7):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

In [10]:
cv = open("Julia_Pikovets_DS.txt", "r").read()

In [11]:
url = 'https://djinni.co/jobs/572624-chatgpt-openai-rd-engineer-data-science-/'

In [12]:
response = urllib.request.urlopen(url)
soup = BeautifulSoup(response,'html.parser',
                         from_encoding=response.info().get_param('charset'))



In [13]:
if soup.find('div', class_='detail--title-wrapper'):
    title = soup.find('div', class_='detail--title-wrapper').get_text().strip()
    print("Title: ", title)
if soup.find('a', class_='job-details--title'):
    company = soup.find('a', class_='job-details--title').get_text().strip()
    print("Company: ", company)
if soup.find('a', class_='job-details--recruiter-name'):
    recruiter = soup.find('a', class_='job-details--recruiter-name').get_text().strip()
    print("Recruiter: ", recruiter)
if soup.find("meta", attrs={"name": "description"}):
    vacancy = soup.find('div', class_='profile-page-section').get_text().strip()
    print("Description :\n", vacancy)

Title:  ChatGPT/OpenAI RD Engineer (Data Science)
Company:  Thelassa.fun
Recruiter:  Yulia Chayka, It Recruiter
Description :
 Наша компанія у пошуках спеціаліста, який допоможе нам реалізувати нові юзкейси на основі генеративних моделей. Шукаємо спеціаліста, який зацікавлений у вивчені ML/AI напрямку, та готовий генерувати ідеї, вивчати інформацію та тренди щодо застосування цих технологій. Наша компанія відкриває набір для заряджених спеціалістів, проактивний та самостійних людей, які мають велике бажання розвиватися у цій ніші. В рамках нашої компанії ми створимо новий стартап на цьому поприщі.Нас цікавить саме створення нових сервісів на основі генеративних моделей. Основна ціль - зробити демо-версії юзкейсів на основі GPT-моделей.Ми міжнародна компанія з розробки ІТ та програмного забезпечення з головним офісом у США. За 6+ років на ринку ми створили стабільну присутність у телекомунікаційному, фінансовому, автомобільному та інших секторах економіки.Досвід:— 1+ рік досвіду програм

In [48]:
prompt = f"""
Write a cover letter with only 200 words in English to recruter {recruiter} using the next instructions:
1. Read my CV and job description which are delimited with triple backticks.
2. Define company name, project description, responsibilities and requirements for {title} position.
3. Determine which technologies and skills are needed in the job description and indicated in the CV.
4. Choose the most relevant duties and experience at work and during other projects based on a comparison of those indicated in the vacancy and in the CV.
5. Based on previous steps write a cover letter. \
Do not repeate details from CV, summarize a specific story that explains why candidate is right for the role. \
Write in a professional, friendly and upbeat tone. \
Make it short.
6. Show only cover letter.
CV: '''{cv}'''
Job description: '''{vacancy}'''
"""
response = get_completion(prompt)
print(response)

Dear Yulia Chayka,

I hope this email finds you well. I recently came across the job posting for the ChatGPT/OpenAI RD Engineer (Data Science) position and I am excited to express my interest in this role.

After carefully reviewing my CV and the job description, I believe that my skills and experience align perfectly with the requirements of the position. My background as a Data Scientist, with expertise in statistics, machine learning, and programming, makes me well-suited for this role. I have extensive experience in training predictive models using Python, as well as implementing end-to-end solutions and conducting A/B testing.

I possess a strong understanding of the technologies and skills mentioned in the job description, including Python, Pytorch, Tensorflow, and MLOps/AIOps best practices. Additionally, I have experience working with Open AI and have a deep understanding of the Python ML ecosystem.

During my previous roles, I have successfully implemented predictive models, c

In [49]:
import re
filename =" ".join(["letter",title, company,".txt"] )
filename = re.sub(r'[\\/*?:"<> |]',"_",filename)
with open(filename, 'a') as f:
    f.write('\n\n')
    f.write(response)